## imports

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

## initialize variables

In [68]:
DEVICE: torch.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{DEVICE = }")

competitor_count = 19  # number of competitors
song_order = np.random.choice(np.arange(1, competitor_count + 1), size=competitor_count, replace=False)
song_order_tensor = torch.from_numpy(song_order).float()
song_order_tensor = song_order_tensor.view(1, -1)
print("Song order:", song_order_tensor)

target_rank = np.random.choice(np.arange(1, competitor_count + 1), size=competitor_count, replace=False)
target_rank_tensor = torch.from_numpy(target_rank).float()
target_rank_tensor = target_rank_tensor.view(1,-1)
print("Target rank:", target_rank_tensor)

DEVICE = device(type='cpu')
Song order: tensor([[14.,  6., 10., 12.,  4.,  9., 18., 16.,  5., 17., 15.,  7.,  8., 13.,
          3.,  1., 11.,  2., 19.]])
Target rank: tensor([[17., 14.,  5.,  2., 11.,  7.,  6., 19., 13.,  1., 12.,  8., 15., 18.,
          4.,  3., 10., 16.,  9.]])


## simple neural network

In [69]:
class LeagueNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int) -> None:
        super(LeagueNN, self).__init__()
        # affine transformations
        self.lin1 = nn.Linear(input_size, hidden_size) 
        # with nonlinearities in between
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(hidden_size, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # apply things in sequence
        out = self.lin1(x.flatten(start_dim=1))
        out = self.relu(out)
        out = self.lin2(out)
        return out


## test model

In [65]:
# instantiate
model = LeagueNN(
    input_size = 19,
    hidden_size = 10,
    output_size = 19
)

# loss
loss_criterion = nn.MSELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01)

epochs = 3000
for epoch in range(epochs):
    # Forward pass
    outputs = model(song_order_tensor)
    
    # Compute the loss
    loss = loss_criterion(outputs, target_rank_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()  # Zero gradients before backward pass
    loss.backward()        # Compute gradients
    optimizer.step()       # Update the weights
    
    if (epoch + 1) % 100 == 0:  # Print loss every 100 epochs
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Test the trained model
with torch.no_grad():  # No need to compute gradients during inference
    predicted = model(song_order_tensor)
    print("Predictions after training:", predicted)
    print("Actual target:", target_rank_tensor)

Epoch [100/3000], Loss: 106.7840
Epoch [200/3000], Loss: 86.5023
Epoch [300/3000], Loss: 70.0727
Epoch [400/3000], Loss: 56.7636
Epoch [500/3000], Loss: 45.9824
Epoch [600/3000], Loss: 37.2489
Epoch [700/3000], Loss: 30.1741
Epoch [800/3000], Loss: 24.4431
Epoch [900/3000], Loss: 19.8005
Epoch [1000/3000], Loss: 16.0398
Epoch [1100/3000], Loss: 12.9933
Epoch [1200/3000], Loss: 10.5255
Epoch [1300/3000], Loss: 8.5263
Epoch [1400/3000], Loss: 6.9069
Epoch [1500/3000], Loss: 5.5951
Epoch [1600/3000], Loss: 4.5324
Epoch [1700/3000], Loss: 3.6715
Epoch [1800/3000], Loss: 2.9742
Epoch [1900/3000], Loss: 2.4093
Epoch [2000/3000], Loss: 1.9517
Epoch [2100/3000], Loss: 1.5810
Epoch [2200/3000], Loss: 1.2807
Epoch [2300/3000], Loss: 1.0375
Epoch [2400/3000], Loss: 0.8404
Epoch [2500/3000], Loss: 0.6808
Epoch [2600/3000], Loss: 0.5515
Epoch [2700/3000], Loss: 0.4468
Epoch [2800/3000], Loss: 0.3619
Epoch [2900/3000], Loss: 0.2932
Epoch [3000/3000], Loss: 0.2375
Predictions after training: tensor([